In [1]:
#default_exp train

# Train the network


In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/pointpillars/")
sys.path.insert(0, "../pointpillars")

from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
#export
import logging
import torch
from torch.utils.tensorboard import SummaryWriter
import datetime
from torchviz import make_dot
from torch.profiler import profile, record_function, ProfilerActivity

from utils.io import read_config, save_network_checkpoint, save_network
from data.dataset import VelTrainDataset, collate_fn, OverfitSampler, collate_wrapper
from modules.pointpillars import PointPillars, init_weights
from modules.loss import PointPillarsLoss


#import os
#os.environ['PATH'] += os.pathsep + "/home/qhs67/anaconda3/envs/ba/lib/python3.8/site-packages/graphviz"


In [4]:
#export
# time to identify the run on folders and checkpoints etc
ident_time = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

In [5]:
#export
# logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
log_formatter = logging.Formatter(log_format)

log_handler = logging.FileHandler("/home/qhs67/git/bachelorthesis_sven_thaele/code/pointpillars.log", mode='w')
log_handler.setFormatter(log_formatter)
logger.addHandler(log_handler)

In [6]:
#export
# tensorboard writer
run_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/runs/{}/".format(ident_time)

writer = SummaryWriter(run_folder)

In [7]:
tens = torch.rand((10,10))
tens.__str__()


'tensor([[0.9286, 0.5153, 0.1014, 0.4156, 0.7509, 0.6023, 0.1398, 0.5218, 0.9959,\n         0.1809],\n        [0.7044, 0.5028, 0.0514, 0.3307, 0.8468, 0.4850, 0.0378, 0.3771, 0.6260,\n         0.6868],\n        [0.8171, 0.3611, 0.0451, 0.8152, 0.4421, 0.7532, 0.9213, 0.5106, 0.5432,\n         0.1461],\n        [0.2137, 0.6373, 0.2152, 0.6954, 0.9749, 0.8745, 0.4388, 0.8018, 0.7215,\n         0.3250],\n        [0.2791, 0.6374, 0.5947, 0.2953, 0.0867, 0.7234, 0.8654, 0.3448, 0.2403,\n         0.2191],\n        [0.2610, 0.9185, 0.9933, 0.1652, 0.8397, 0.6247, 0.3722, 0.2197, 0.1521,\n         0.4104],\n        [0.0108, 0.1410, 0.5190, 0.0044, 0.1795, 0.9898, 0.4361, 0.3983, 0.7308,\n         0.2474],\n        [0.0116, 0.4330, 0.9442, 0.3807, 0.5948, 0.3696, 0.3961, 0.3705, 0.9404,\n         0.3496],\n        [0.5895, 0.4239, 0.3905, 0.4346, 0.2233, 0.3486, 0.2616, 0.8417, 0.8531,\n         0.6779],\n        [0.5821, 0.8227, 0.6498, 0.4859, 0.8083, 0.3821, 0.1802, 0.5034, 0.9351,\n        

In [8]:
#export
def _train_setup():
    """

    """
    batch_size = 3
    init_lr = 2 * 10**-4
    #init_lr = 1 * 10**-4

    logger.info("Start network training..")
    torch.cuda.empty_cache()
    #torch.multiprocessing.set_start_method('spawn')
    # TODO: move to config file
    conf = read_config()
    vel_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/training"
    label_folder ="/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/training"
    ds_train = VelTrainDataset(vel_folder, label_folder)
    dl_train = torch.utils.data.DataLoader(ds_train,
                                           batch_size=batch_size,
                                           num_workers=2,
                                           collate_fn=collate_wrapper,
                                           pin_memory=True,
                                           shuffle=True)
    """sampler = OverfitSampler(ds_train, batch_size, nb_samples=20, shuffle=True)
    for i in sampler:
        print(i)
    dl_train = torch.utils.data.DataLoader(ds_train,
                                           batch_size=batch_size,
                                           sampler=sampler,
                                           num_workers=0,
                                           collate_fn=collate_fn)"""
    # modules
    pointpillars = PointPillars(conf)
    loss_func = PointPillarsLoss()
    pointpillars.train()
    loss_func.train()

    # TODO: also init bias?
    #pointpillars.apply(init_weights)
    # move to gpu
    pointpillars.cuda()
    loss_func.cuda()

    optimizer = torch.optim.Adam(pointpillars.parameters(), lr=init_lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, gamma=0.8, last_epoch=-1)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 2000, gamma=0.8, last_epoch=-1)

    return pointpillars, loss_func, optimizer, scheduler, dl_train

In [9]:
#export
def _train_step(batch: torch.Tensor,
               pointpillars: torch.nn.Module,
               loss_func: torch.nn.Module,
               optimizer: torch.optim.Adam,
               epoch: int,
               i: int) -> torch.nn.Module:
    """
    Performs a training step
    """
    pil_batch, ind_batch, label_batch, label_mask = batch

    pil_batch = pil_batch.cuda(non_blocking=True)
    ind_batch = ind_batch.cuda(non_blocking=True)
    label_batch = label_batch.cuda(non_blocking=True)
    label_mask = label_mask.cuda(non_blocking=True)
    # -> forward pass through network

    preds = pointpillars(pil_batch, ind_batch, label_batch, label_mask)
    loss = loss_func(preds, writer, epoch, i)

    """dot = make_dot(loss, params=dict(pointpillars.named_parameters()), show_saved=True, show_attrs=True)
    dot.format = 'png'
    dot.render('torchviz-sample')"""
    loss.backward()
    optimizer.step()

    del pil_batch, ind_batch, label_batch, preds

    return loss

In [10]:
#export
def validate(network: torch.nn.Module, loss_func: torch.nn.Module, epoch, nbr_val_batches: int = 300):
    """
    Validates the network on the loss function on the validation dataset
    """
    batch_size = 3
    validation_folder = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/velodyne/validation"
    label_folder ="/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/label_2/validation"
    with torch.no_grad():
        ds_val = VelTrainDataset(validation_folder, label_folder)
        dl_val = torch.utils.data.DataLoader(ds_val,
                                           batch_size=batch_size,
                                           num_workers=2,
                                           collate_fn=collate_fn,
                                           pin_memory=True,
                                           shuffle=True)

        running_val_loss = 0
        for i, batch in enumerate(dl_val):
            # stop after given number of data
            if i >= nbr_val_batches:
                break
            pil_batch, ind_batch, label_batch, label_mask = batch
            preds = network(pil_batch, ind_batch, label_batch, label_mask)
            loss = loss_func(preds)
            running_val_loss += loss.item()

            print("Val Epoch: {}, Batch {} with Loss {} and running Loss {}.".format(epoch, i, loss.item(), running_val_loss/(i+1)))
            #writer.add_scalar("Epoch {}/Validation Loss".format(epoch), running_val_loss/(i+1), i)
            #writer.flush()

        writer.add_scalar("Epochs/Validation Loss", running_val_loss/nbr_val_batches, epoch)
        writer.flush()


In [11]:
#export
def train(val: bool = False, save_nw: bool = False):
    """

    :param val: bool if validation should be used
    :param save_nw: bool if network state should be saved after training
    """
    n_epochs = 1

    pointpillars, loss_func, optimizer, scheduler, dl_train = _train_setup()


    try:
        for epoch in range(n_epochs):
            running_loss = 0
            with torch.profiler.profile(
                schedule=torch.profiler.schedule(wait=0, warmup=1, active=3, repeat=2),
                on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/prof1'),
                record_shapes=True,
                with_stack=True,

            ) as prof:
                for i, batch in enumerate(dl_train):
                    if i >= (1 + 3) * 2:
                        break
                    optimizer.zero_grad()
                    loss = _train_step(batch, pointpillars, loss_func, optimizer, epoch, i)
                    #running_loss += loss.item()
                    #print("Epoch: {}, Batch {} with Loss {} and running Loss {}.".format(epoch, i, loss.item(), running_loss/(i+1)))
                    print("Epoch: {}, Batch {}.".format(epoch, i))
                    #writer.add_scalar("Epoch {}/Running Loss".format(epoch), running_loss/(i+1), i)
                    #writer.flush()
                    prof.step()

                # after epoch
                #scheduler.step()
                #writer.add_scalar("Epochs/Running Loss", running_loss/len(dl_train), epoch)
                #writer.add_scalar("Epochs/Learning Rate", scheduler.get_last_lr()[0], epoch)
                #writer.flush()
            #if val:
                #validate(pointpillars, loss_func, epoch, nbr_val_batches=100)
        # after training
        print('Finished Training')

    except Exception as e:
        print(e)
        logger.exception("An exception occured")
        save_network_checkpoint(pointpillars, optimizer, scheduler, loss, ident_time, epoch)
        exit()
    if save_nw:
        # save network to location from config
        save_network(pointpillars, ident_time, n_epochs)


    writer.close()

In [12]:
a = torch.tensor([[[0, 1, 2], [10, 20, 30]], [[100, 200, 300], [1000, 2000, 3000]]])
a, a.shape

(tensor([[[   0,    1,    2],
          [  10,   20,   30]],
 
         [[ 100,  200,  300],
          [1000, 2000, 3000]]]),
 torch.Size([2, 2, 3]))

In [13]:
b = torch.flatten(a, start_dim=0, end_dim=1)
b, b.shape, b[2]

(tensor([[   0,    1,    2],
         [  10,   20,   30],
         [ 100,  200,  300],
         [1000, 2000, 3000]]),
 torch.Size([4, 3]),
 tensor([100, 200, 300]))

In [14]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_cached())
#train(val=True, save_nw=False)
#make_dot(loss, params=dict(pointpillars.named_parameters()))

0
0


/home/qhs67/anaconda3/envs/ba/lib/python3.8/site-packages/torch/cuda/memory.py:373: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [ ]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_interference"):
        train(save_nw=False)
        
    print(prof.key_averages().table())

Epoch: 0, Batch 0 with Loss 2.6619033813476562 and running Loss 2.6619033813476562.
Epoch: 0, Batch 1 with Loss 2.680222988128662 and running Loss 2.671063184738159.


In [ ]:
#export
if __name__ == '__main__':
    train(save_nw=False)


